# Preliminaries
We need to load a few libraries first

In [94]:
import pandas as pd
import numpy as np
import ampligraph
import stardog
import json
import io
print(ampligraph.__version__)

2.0.0


# Loading the data

In [95]:
df = pd.read_csv("graph_content.csv.gz").dropna()
df.head()

,s,p,o
0,tag:stardog:designer:Tutorial:data:Match:3249,tag:stardog:designer:Tutorial:model:match_team,tag:stardog:designer:Tutorial:data:Team:8472
1,tag:stardog:designer:Tutorial:data:Match:3249,tag:stardog:designer:Tutorial:model:match_team,tag:stardog:designer:Tutorial:data:Team:9825
2,tag:stardog:designer:Tutorial:data:Match:3250,tag:stardog:designer:Tutorial:model:match_team,tag:stardog:designer:Tutorial:data:Team:10260
3,tag:stardog:designer:Tutorial:data:Match:3250,tag:stardog:designer:Tutorial:model:match_team,tag:stardog:designer:Tutorial:data:Team:8668
4,tag:stardog:designer:Tutorial:data:Match:3251,tag:stardog:designer:Tutorial:model:match_team,tag:stardog:designer:Tutorial:data:Team:9850


# Train the model

In [96]:
from ampligraph.evaluation import train_test_split_no_unseen 
test_size = int(df.shape[0]*0.10)
X_train, X_valid = train_test_split_no_unseen(np.array(df), test_size=test_size)
print('Train set size: ', X_train.shape)
print('Test set size: ', X_valid.shape)

Train set size:  (9090, 3)
Test set size:  (1010, 3)


In [97]:
from ampligraph.latent_features import ScoringBasedEmbeddingModel
from ampligraph.latent_features.regularizers import get as get_regularizer
from ampligraph.latent_features.loss_functions import get as get_loss
import tensorflow as tf

# create the model
model = ScoringBasedEmbeddingModel(k=200, eta=10, 
                                   scoring_type='ComplEx', seed=0)

# create reg, loss, optim
reg = get_regularizer('LP', hyperparams={'p':1, 'lambda':1e-2})
loss = get_loss('pairwise', hyperparams={'margin':0.5})
optim = tf.keras.optimizers.Adam(learning_rate = 1e-4)

# compile the model
model.compile(loss=loss, optimizer=optim, entity_relation_regularizer=reg)
model.fit(X_train, batch_size=2000, epochs=800)

Epoch 1/800
6/6 [==============================] - 1s 162ms/step - loss: 9393.8867
Epoch 2/800
6/6 [==============================] - 0s 33ms/step - loss: 9390.3975
Epoch 3/800
6/6 [==============================] - 0s 33ms/step - loss: 9387.1035
Epoch 4/800
6/6 [==============================] - 0s 32ms/step - loss: 9383.8848
Epoch 5/800
6/6 [==============================] - 0s 32ms/step - loss: 9380.6855
Epoch 6/800
6/6 [==============================] - 0s 31ms/step - loss: 9377.5244
Epoch 7/800
6/6 [==============================] - 0s 32ms/step - loss: 9374.4004
Epoch 8/800
6/6 [==============================] - 0s 32ms/step - loss: 9371.3145
Epoch 9/800
6/6 [==============================] - 0s 31ms/step - loss: 9368.2412
Epoch 10/800
6/6 [==============================] - 0s 32ms/step - loss: 9365.2080
Epoch 11/800
6/6 [==============================] - 0s 32ms/step - loss: 9362.1943
Epoch 12/800
6/6 [==============================] - 0s 34ms/step - loss: 9359.1924
Epoch 13/800

Epoch 233/800
6/6 [==============================] - 0s 29ms/step - loss: 5647.2969
Epoch 234/800
6/6 [==============================] - 0s 32ms/step - loss: 5627.1523
Epoch 235/800
6/6 [==============================] - 0s 31ms/step - loss: 5607.1934
Epoch 236/800
6/6 [==============================] - 0s 38ms/step - loss: 5587.3936
Epoch 237/800
6/6 [==============================] - 0s 36ms/step - loss: 5567.7593
Epoch 238/800
6/6 [==============================] - 0s 34ms/step - loss: 5548.2588
Epoch 239/800
6/6 [==============================] - 0s 33ms/step - loss: 5528.9312
Epoch 240/800
6/6 [==============================] - 0s 31ms/step - loss: 5509.7651
Epoch 241/800
6/6 [==============================] - 0s 31ms/step - loss: 5490.7695
Epoch 242/800
6/6 [==============================] - 0s 31ms/step - loss: 5471.8740
Epoch 243/800
6/6 [==============================] - 0s 32ms/step - loss: 5453.1479
Epoch 244/800
6/6 [==============================] - 0s 31ms/step - loss: 54

In [63]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score
ranks = model.evaluate(X_valid, batch_size=2000, verbose=True)
print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('Hits@1:', hits_at_n_score(ranks, 1))
print('Hits@3:', hits_at_n_score(ranks, 3))
print('Hits@10:', hits_at_n_score(ranks, 10))
print('Hits@100:', hits_at_n_score(ranks, 100))

2023-04-04 11:17:15.393515: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: RaggedFromRowLengths/RowPartitionFromRowLengths/assert_non_negative/assert_less_equal/Assert/AssertGuard/branch_executed/_8


2/2 [==============================] - 6s 3s/step
MR: 698.7544554455445
MRR: 0.06574416555862453
Hits@1: 0.0
Hits@3: 0.08316831683168317
Hits@10: 0.1608910891089109
Hits@100: 0.38811881188118813


# Make predictions

We will now generate all the pairs of <player, plays for, team> and rank them by prediction score

In [64]:
players = set(df[df['s'].str.contains('Player:')]['s'].values) | \
          set(df[df['o'].str.contains('Player:')]['o'].values)
teams = set(df[df['s'].str.contains('Team:')]['s'].values) | \
        set(df[df['o'].str.contains('Team:')]['o'].values)
print('Number of players: ', len(players))
print('Number of teams: ', len(teams))

Number of players:  2344
Number of teams:  98


In [65]:
pairs = []
for player in players:
  for team in teams:
    pairs.append([player, 'tag:stardog:designer:Tutorial:model:plays_for', team])
# Create the possible combinations to test
df_to_test = pd.DataFrame(pairs, columns=['s', 'p', 'o'])
# Remove all the pairs we used for training
df_to_test = pd.merge(df_to_test, df, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
print (df_to_test.shape)
df_to_test.head()

(227262, 3)


,s,p,o
0,tag:stardog:designer:Tutorial:data:Player:39447,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8654
1,tag:stardog:designer:Tutorial:data:Player:39447,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9864
2,tag:stardog:designer:Tutorial:data:Player:39447,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9783
4,tag:stardog:designer:Tutorial:data:Player:39447,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8529
5,tag:stardog:designer:Tutorial:data:Player:39447,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8551


In [66]:
df_to_test['scores'] = model.predict(np.array(df_to_test))
df_to_test = df_to_test.sort_values(by='scores', ascending=False)
df_to_test

,s,p,o,scores
65209,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8633,0.511274
229018,tag:stardog:designer:Tutorial:data:Player:41497,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8540,0.462421
88682,tag:stardog:designer:Tutorial:data:Player:42041,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8540,0.437658
49507,tag:stardog:designer:Tutorial:data:Player:30893,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8634,0.406167
199485,tag:stardog:designer:Tutorial:data:Player:31037,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9941,0.387238
...,...,...,...,...
187125,tag:stardog:designer:Tutorial:data:Player:421785,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9790,-0.194937
72095,tag:stardog:designer:Tutorial:data:Player:289252,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8371,-0.204001
213095,tag:stardog:designer:Tutorial:data:Player:149895,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9790,-0.210471
213108,tag:stardog:designer:Tutorial:data:Player:149895,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8315,-0.222462


In [92]:
df_to_test[df_to_test.s == 'tag:stardog:designer:Tutorial:data:Player:30981']

,s,p,o,scores
65209,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8633,0.511274
65190,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9906,0.281800
65183,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:7878,0.279305
65200,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9910,0.264320
65226,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8315,0.254224
...,...,...,...,...
65185,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8636,-0.077301
65265,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9827,-0.080506
65248,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9851,-0.092022
65233,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9825,-0.107957


# Improve the presentation
Figuring things out based on URIs is a bit challenging. We will return to Stardog to ask for the label associated to each of those identifiers and update the data frame with those.

For running the following cells you will need to create a file `connection_details.json` with this content in it:

```json
{
  "database": "YOUR DATABASE (e.g. Tutorial)",
  "endpoint": "YOUR ENDPOINT (e.g. https://sd-0fcedc52.stardog.cloud:5820)",
  "username": "YOUR USERNAME (e.g. christophe.gueret@accenture.com)",
  "password": "YOUR PASSWORD (e.g. Passw0rd)",
}
```

In [67]:
connection_details = json.load(open('connection_details.json'))
conn = stardog.Connection(**connection_details)

In [89]:
query = """
SELECT ?identifier ?label FROM stardog:context:all WHERE {
  ?identifier rdfs:label ?label .
}
"""
csv_results = conn.select(query, content_type='text/csv')
df_labels = pd.read_csv(io.BytesIO(csv_results)).set_index('identifier')
labels = df_labels.to_dict()['label']
print ('Loaded {} labels'.format(len(labels)))

Loaded 2673 labels


In [91]:
df_to_test.replace(labels)

,s,p,o,scores
65209,Lionel Messi,plays for,Real Madrid CF,0.511274
229018,Federico Peluso,plays for,Palermo,0.462421
88682,Emanuele Calaio,plays for,Palermo,0.437658
49507,Cristiano Ronaldo,plays for,FC Barcelona,0.406167
199485,Jacques Alaixys Romao,plays for,Toulouse FC,0.387238
...,...,...,...,...
187125,Andre Auras,plays for,Hamburger SV,-0.194937
72095,Timothee Dieng,plays for,CA Osasuna,-0.204001
213095,Jordan Bowery,plays for,Hamburger SV,-0.210471
213108,Jordan Bowery,plays for,Athletic Club de Bilbao,-0.222462


In [93]:
df_to_test[df_to_test.s == 'tag:stardog:designer:Tutorial:data:Player:30981'].replace(labels)

,s,p,o,scores
65209,Lionel Messi,plays for,Real Madrid CF,0.511274
65190,Lionel Messi,plays for,Atlético Madrid,0.281800
65183,Lionel Messi,plays for,Granada CF,0.279305
65200,Lionel Messi,plays for,RC Celta de Vigo,0.264320
65226,Lionel Messi,plays for,Athletic Club de Bilbao,0.254224
...,...,...,...,...
65185,Lionel Messi,plays for,Inter,-0.077301
65265,Lionel Messi,plays for,Girondins de Bordeaux,-0.080506
65248,Lionel Messi,plays for,Stade Rennais FC,-0.092022
65233,Lionel Messi,plays for,Arsenal,-0.107957
